In [136]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

data = pd.read_csv(r"D:\FraudDetection\fraudTrain.csv")
data.dropna(inplace=True)
data.info()
test = pd.read_excel(r"D:\FraudDetection\fraudTest.xlsx")
test.dropna(inplace=True)



<class 'pandas.core.frame.DataFrame'>
Index: 1002103 entries, 0 to 1002102
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   ID           1002103 non-null  int64  
 1   Time         1002103 non-null  object 
 2   Card Number  1002103 non-null  float64
 3   merchant     1002103 non-null  object 
 4   category     1002103 non-null  object 
 5   Amount       1002103 non-null  float64
 6   firstName    1002103 non-null  object 
 7   lastName     1002103 non-null  object 
 8   trans_num    1002103 non-null  object 
 9   is_fraud     1002103 non-null  float64
dtypes: float64(3), int64(1), object(6)
memory usage: 84.1+ MB


In [137]:
data['Time'] = pd.to_datetime(data['Time'])
test['Time'] = pd.to_datetime(test['Time'])



data.head()


,ID,Time,Card Number,merchant,category,Amount,firstName,lastName,trans_num,is_fraud
0,0,2019-01-01 00:00:00,2.700000e+15,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,0b242abb623afc578575680df30655b9,0.0
1,1,2019-01-01 00:00:00,6.300000e+11,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,1f76529f8574734946361c461b024d99,0.0
2,2,2019-01-01 00:00:00,3.890000e+13,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,a1a22d70485983eac12b5b88dad1cf95,0.0
3,3,2019-01-01 00:01:00,3.530000e+15,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,6b849c168bdad6f867558c3793159a81,0.0
4,4,2019-01-01 00:03:00,3.760000e+14,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,a41d7549acf90789359a9aa5346dcb46,0.0


# NEW COLUMNS

In [138]:
data['year'] = data['Time'].dt.year
data['day'] = data['Time'].dt.day
data['month'] = data['Time'].dt.month
data['hour'] = data['Time'].dt.hour

data['fullName'] = data['firstName'] + data['lastName']
data['fullNamehour'] = data['fullName'] + data['year'].astype(str) + data['month'].astype(str) + data['day'].astype(str) + data['hour'].astype(str)
data['fullNamesinglehour'] = data['fullName'] + data['hour'].astype(str)
data['fullNamecard'] = data['fullName'] + data['Card Number'].astype(str)
data['cardhour'] = data['Card Number'].astype(str) + data['year'].astype(str) + data['month'].astype(str) + data['day'].astype(str) + data['hour'].astype(str)
data['fullNameday'] = data['fullName'] + data['year'].astype(str) + data['month'].astype(str) + data['day'].astype(str)

###

test['year'] = test['Time'].dt.year
test['day'] = test['Time'].dt.day
test['month'] = test['Time'].dt.month
test['hour'] = test['Time'].dt.hour

test['fullName'] = test['firstName'] + test['lastName']
test['fullNamehour'] = test['fullName'] + test['year'].astype(str) + test['month'].astype(str) + test['day'].astype(str) + test['hour'].astype(str)
test['fullNamesinglehour'] = test['fullName'] + test['hour'].astype(str)
test['fullNamecard'] = test['fullName'] + test['Card Number'].astype(str)
test['cardhour'] = test['Card Number'].astype(str) + test['year'].astype(str) + test['month'].astype(str) + test['day'].astype(str) + test['hour'].astype(str)
test['fullNameday'] = test['fullName'] + test['year'].astype(str) + test['month'].astype(str) + test['day'].astype(str)


###########################################################################

# data = data.sort_values(by=['Card Number', 'Time'])
# data['time_diff'] = data.groupby('Card Number')['Time'].diff().fillna(pd.Timedelta(seconds=0))

# ###

# test = test.sort_values(by=['Card Number', 'Time'])
# test['time_diff'] = test.groupby('Card Number')['Time'].diff().fillna(pd.Timedelta(seconds=0))


###########################################################################



counts = data['fullName'].value_counts()
data['countName'] = data['fullName'].map(counts)

counts = data['fullNamehour'].value_counts()
data['countNamehour'] = data['fullNamehour'].map(counts)

counts = data['fullNamesinglehour'].value_counts()
data['countNamesinglehour'] = data['fullNamesinglehour'].map(counts)

counts = data['fullNamecard'].value_counts()
data['countNamecard'] = data['fullNamecard'].map(counts)


counts = data['cardhour'].value_counts()
data['countcardhour'] = data['cardhour'].map(counts)



###

counts = test['fullName'].value_counts()
test['countName'] = test['fullName'].map(counts)

counts = test['fullNamehour'].value_counts()
test['countNamehour'] = test['fullNamehour'].map(counts)

counts = test['fullNamesinglehour'].value_counts()
test['countNamesinglehour'] = test['fullNamesinglehour'].map(counts)

counts = test['fullNamecard'].value_counts()
test['countNamecard'] = test['fullNamecard'].map(counts)



counts = test['cardhour'].value_counts()
test['countcardhour'] = test['cardhour'].map(counts)

############################################################################

data['dis'] = data['Amount'] - data['Amount'].mean()

###

test['dis'] = test['Amount'] - test['Amount'].mean()


############################################################################





data.head()

,ID,Time,Card Number,merchant,category,Amount,firstName,lastName,trans_num,is_fraud,...,fullNamesinglehour,fullNamecard,cardhour,fullNameday,countName,countNamehour,countNamesinglehour,countNamecard,countcardhour,dis
0,0,2019-01-01 00:00:00,2.700000e+15,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,0b242abb623afc578575680df30655b9,0.0,...,JenniferBanks0,JenniferBanks2700000000000000.0,2700000000000000.02019110,JenniferBanks201911,1578,1,40,1578,1,-65.286157
1,1,2019-01-01 00:00:00,6.300000e+11,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,1f76529f8574734946361c461b024d99,0.0,...,StephanieGill0,StephanieGill630000000000.0,630000000000.02019110,StephanieGill201911,2363,1,62,2363,3,36.973843
2,2,2019-01-01 00:00:00,3.890000e+13,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,a1a22d70485983eac12b5b88dad1cf95,0.0,...,EdwardSanchez0,EdwardSanchez38900000000000.0,38900000000000.02019110,EdwardSanchez201911,379,1,19,379,1,149.853843
3,3,2019-01-01 00:01:00,3.530000e+15,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,6b849c168bdad6f867558c3793159a81,0.0,...,JeremyWhite0,JeremyWhite3530000000000000.0,3530000000000000.02019110,JeremyWhite201911,382,1,15,382,2,-25.256157
4,4,2019-01-01 00:03:00,3.760000e+14,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,a41d7549acf90789359a9aa5346dcb46,0.0,...,TylerGarcia0,TylerGarcia376000000000000.0,376000000000000.02019110,TylerGarcia201911,1570,1,57,1570,1,-28.296157


# DROP COLUMNS

In [139]:
data.drop(['Card Number','day','year','fullName','fullNamehour','fullNamesinglehour','fullNamecard','cardhour','fullNameday'],axis=1,inplace=True)
data.drop(['Time','ID','merchant','trans_num','firstName','lastName'],axis=1,inplace=True)

##


test.drop(['Card Number','day','year','fullName','fullNamehour','fullNamesinglehour','fullNamecard','cardhour','fullNameday'],axis=1,inplace=True)
test.drop(['Time','ID','merchant','trans_num','firstName','lastName'],axis=1,inplace=True)



In [140]:
data.describe()

,Amount,is_fraud,month,hour,countName,countNamehour,countNamesinglehour,countNamecard,countcardhour,dis
count,1.002103e+06,1.002103e+06,1.002103e+06,1.002103e+06,1.002103e+06,1.002103e+06,1.002103e+06,1.002103e+06,1.002103e+06,1.002103e+06
mean,7.025616e+01,5.701011e-03,6.700486e+00,1.280453e+01,1.421049e+03,1.230413e+00,6.760443e+01,1.405742e+03,2.739542e+00,-2.868879e-14
std,1.602039e+02,7.528954e-02,3.645710e+00,6.816683e+00,5.910044e+02,5.226107e-01,3.814408e+01,5.733059e+02,2.827608e+00,1.602039e+02
min,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,7.000000e+00,1.000000e+00,1.000000e+00,7.000000e+00,1.000000e+00,-6.925616e+01
25%,9.640000e+00,0.000000e+00,3.000000e+00,7.000000e+00,1.129000e+03,1.000000e+00,3.900000e+01,1.089000e+03,1.000000e+00,-6.061616e+01
50%,4.742000e+01,0.000000e+00,7.000000e+00,1.400000e+01,1.531000e+03,1.000000e+00,5.900000e+01,1.518000e+03,2.000000e+00,-2.283616e+01
75%,8.304000e+01,0.000000e+00,1.000000e+01,1.900000e+01,1.953000e+03,1.000000e+00,8.900000e+01,1.942000e+03,3.000000e+00,1.278384e+01
max,2.894890e+04,1.000000e+00,1.200000e+01,2.300000e+01,3.524000e+03,8.000000e+00,2.000000e+02,2.451000e+03,4.000000e+01,2.887864e+04


In [141]:
test.describe()

,Amount,is_fraud,month,hour,countName,countNamehour,countNamesinglehour,countNamecard,countcardhour,dis
count,555719.000000,555719.000000,555719.000000,555719.000000,555719.000000,555719.000000,555719.000000,555719.000000,555719.000000,5.557190e+05
mean,69.392810,0.003860,9.508536,12.809062,790.149221,1.243321,38.125976,781.478364,1.240856,5.760864e-16
std,156.745941,0.062008,1.978205,6.810924,333.555220,0.528449,22.103919,323.541973,0.525007,1.567459e+02
min,1.000000,0.000000,6.000000,0.000000,6.000000,1.000000,1.000000,6.000000,1.000000,-6.839281e+01
25%,9.630000,0.000000,8.000000,7.000000,611.000000,1.000000,22.000000,503.000000,1.000000,-5.976281e+01
50%,47.290000,0.000000,10.000000,14.000000,827.000000,1.000000,33.000000,821.000000,1.000000,-2.210281e+01
75%,83.010000,0.000000,12.000000,19.000000,1070.000000,1.000000,50.000000,1057.000000,1.000000,1.361719e+01
max,22768.110000,1.000000,12.000000,23.000000,1965.000000,8.000000,130.000000,1474.000000,8.000000,2.269872e+04


In [142]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1002103 entries, 0 to 1002102
Data columns (total 11 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   category             1002103 non-null  object 
 1   Amount               1002103 non-null  float64
 2   is_fraud             1002103 non-null  float64
 3   month                1002103 non-null  int32  
 4   hour                 1002103 non-null  int32  
 5   countName            1002103 non-null  int64  
 6   countNamehour        1002103 non-null  int64  
 7   countNamesinglehour  1002103 non-null  int64  
 8   countNamecard        1002103 non-null  int64  
 9   countcardhour        1002103 non-null  int64  
 10  dis                  1002103 non-null  float64
dtypes: float64(3), int32(2), int64(5), object(1)
memory usage: 84.1+ MB


In [143]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   category             555719 non-null  object 
 1   Amount               555719 non-null  float64
 2   is_fraud             555719 non-null  int64  
 3   month                555719 non-null  int32  
 4   hour                 555719 non-null  int32  
 5   countName            555719 non-null  int64  
 6   countNamehour        555719 non-null  int64  
 7   countNamesinglehour  555719 non-null  int64  
 8   countNamecard        555719 non-null  int64  
 9   countcardhour        555719 non-null  int64  
 10  dis                  555719 non-null  float64
dtypes: float64(2), int32(2), int64(6), object(1)
memory usage: 42.4+ MB


# OutLiers

In [144]:
# q1 = data['Amount'].quantile(0.25)
# q3 = data['Amount'].quantile(0.75)
# iqr = q3 - q1

# lower = q1 - 1.5*iqr
# upper = q3 + 1.5*iqr



# # upper = data['Amount'].mean() + 3*data['Amount'].std()
# # lower = data['Amount'].mean() - 3*data['Amount'].std()

# print(upper," ",lower)

In [145]:
# # data = data.loc[(data['Amount'] > lower) & (data['Amount'] < upper)]

# outliers1 = (data['Amount'] < lower)
# data['Amount'] = data['Amount'].where(~outliers1, other=lower)

# outliers2 = (data['Amount'] > upper)
# data['Amount'] = data['Amount'].where(~outliers2, other=upper)

# data.describe()


In [146]:

# q1 = test['Amount'].quantile(0.25)
# q3 = test['Amount'].quantile(0.75)

# iqr = q3 - q1

# lower = q1 - 1.5*iqr
# upper = q3 + 1.5*iqr

# outliers1 = (test['Amount'] < lower)
# test['Amount'] = test['Amount'].where(~outliers1, other=lower)

# outliers2 = (test['Amount'] > upper)
# test['Amount'] = test['Amount'].where(~outliers2, other=upper)
# test.describe()

# Label Encoding

In [147]:
label_encoder = LabelEncoder()

data['category'] = label_encoder.fit_transform(data['category'])
# data['time_diff'] = label_encoder.fit_transform(data['time_diff'])




test['category'] = label_encoder.fit_transform(test['category'])
# test['time_diff'] = label_encoder.fit_transform(test['time_diff'])

data.head()


,category,Amount,is_fraud,month,hour,countName,countNamehour,countNamesinglehour,countNamecard,countcardhour,dis
0,8,4.97,0.0,1,0,1578,1,40,1578,1,-65.286157
1,4,107.23,0.0,1,0,2363,1,62,2363,3,36.973843
2,0,220.11,0.0,1,0,379,1,19,379,1,149.853843
3,2,45.00,0.0,1,0,382,1,15,382,2,-25.256157
4,9,41.96,0.0,1,0,1570,1,57,1570,1,-28.296157


In [148]:
data.corr()

,category,Amount,is_fraud,month,hour,countName,countNamehour,countNamesinglehour,countNamecard,countcardhour,dis
category,1.000000,0.030492,0.019908,0.000812,0.157480,0.005569,0.031895,0.139970,0.007059,0.028177,0.030492
Amount,0.030492,1.000000,0.217171,-0.001747,-0.022993,-0.009344,0.018648,-0.029964,-0.009285,0.002846,1.000000
is_fraud,0.019908,0.217171,1.000000,-0.014178,0.012535,-0.051202,0.088604,-0.047118,-0.052029,0.015497,0.217171
month,0.000812,-0.001747,-0.014178,1.000000,0.000609,0.000040,0.093378,0.000981,0.000050,0.136385,-0.001747
hour,0.157480,-0.022993,0.012535,0.000609,1.000000,0.106775,0.089699,0.399367,0.109740,0.095799,-0.022993
countName,0.005569,-0.009344,-0.051202,0.000040,0.106775,1.000000,0.190380,0.787381,0.969388,0.077968,-0.009344
countNamehour,0.031895,0.018648,0.088604,0.093378,0.089699,0.190380,1.000000,0.235367,0.185745,0.281671,0.018648
countNamesinglehour,0.139970,-0.029964,-0.047118,0.000981,0.399367,0.787381,0.235367,1.000000,0.771963,0.116314,-0.029964
countNamecard,0.007059,-0.009285,-0.052029,0.000050,0.109740,0.969388,0.185745,0.771963,1.000000,0.086411,-0.009285
countcardhour,0.028177,0.002846,0.015497,0.136385,0.095799,0.077968,0.281671,0.116314,0.086411,1.000000,0.002846


In [149]:
test.corr()

,category,Amount,is_fraud,month,hour,countName,countNamehour,countNamesinglehour,countNamecard,countcardhour,dis
category,1.000000,0.026811,0.016980,0.000264,0.157214,0.005399,0.034513,0.136332,0.006453,0.034962,0.026811
Amount,0.026811,1.000000,0.182267,-0.000717,-0.029860,-0.007849,0.011397,-0.026590,-0.007715,0.011546,1.000000
is_fraud,0.016980,0.182267,1.000000,-0.011748,0.011686,-0.037617,0.073481,-0.031670,-0.038545,0.073924,0.182267
month,0.000264,-0.000717,-0.011748,1.000000,-0.004995,0.001507,0.072873,-0.001953,0.000906,0.071942,-0.000717
hour,0.157214,-0.029860,0.011686,-0.004995,1.000000,0.111958,0.097679,0.388996,0.114440,0.097501,-0.029860
countName,0.005399,-0.007849,-0.037617,0.001507,0.111958,1.000000,0.208241,0.779708,0.969606,0.198612,-0.007849
countNamehour,0.034513,0.011397,0.073481,0.072873,0.097679,0.208241,1.000000,0.260739,0.203886,0.994175,0.011397
countNamesinglehour,0.136332,-0.026590,-0.031670,-0.001953,0.388996,0.779708,0.260739,1.000000,0.764211,0.254624,-0.026590
countNamecard,0.006453,-0.007715,-0.038545,0.000906,0.114440,0.969606,0.203886,0.764211,1.000000,0.205163,-0.007715
countcardhour,0.034962,0.011546,0.073924,0.071942,0.097501,0.198612,0.994175,0.254624,0.205163,1.000000,0.011546


# Model Training

In [150]:


data.to_csv("processedTrain.csv")
test.to_csv("processedTest.csv")


# scaler = MinMaxScaler()

# numerical_features = ['Amount','month','hour','countName','countNamehour','countNamesinglehour','countNamecard','countcardhour','dis']

# test[numerical_features] = scaler.fit_transform(test[numerical_features])
# data[numerical_features] = scaler.fit_transform(data[numerical_features])



# X = data.drop(['is_fraud'],axis=1)
# y = data['is_fraud']

# X_train = X
# X_test = test.drop(['is_fraud'],axis=1)
# y_train = y
# y_test = test['is_fraud']



# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)


# X_test = test.drop(['is_fraud'],axis=1)
# y_test = test['is_fraud']


########################################################3

# svm_classifier = SVC(kernel='linear')  



################################################


# clf = DecisionTreeClassifier()



##########################################################
 #svmmodel = DecisionTreeClassifier()



In [151]:
# from sklearn.metrics import confusion_matrix
# from sklearn.linear_model import LogisticRegression
# import xgboost as xgb

# # model = RandomForestClassifier(n_estimators=100, random_state=42)
# # model = LogisticRegression()
# model = xgb.XGBClassifier(objective='multi:softmax', num_class=3, random_state=42)


# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)




# accuracy = accuracy_score(y_test, y_pred)
# report = classification_report(y_test, y_pred)

# # confussion matrix (DONE)
# # oversampling

# print("Accuracy:", accuracy)
# print("Classification Report:\n", report)
# cmRandom = confusion_matrix(y_test, y_pred)

# print("Confusion Matrix:")
# print(cmRandom)



In [152]:
# Logisticmodel = LogisticRegression()

# Logisticmodel.fit(X_train, y_train)

# y_predLogistic = Logisticmodel.predict(X_test)




# cmLogistic = confusion_matrix(y_test, y_predLogistic)
# accuracylogistic = accuracy_score(y_test, y_predLogistic)
# logisticreport = classification_report(y_test, y_predLogistic)

# # confussion matrix (DONE)
# # oversampling

# print("Accuracy:", accuracylogistic)
# print("Classification Report:\n", logisticreport)
# print("Confusion Matrix:")
# print(cmLogistic)

In [153]:
# import xgboost as xgb

# xgboostmodel = xgb.XGBClassifier(objective='multi:softmax', num_class=3, random_state=42)

# xgboostmodel.fit(X_train, y_train)

# y_predxgboost = xgboostmodel.predict(X_test)

# accuracyxgbboost = accuracy_score(y_test, y_predxgboost)
# xgboostreport = classification_report(y_test, y_predxgboost)


# print("Accuracy:", accuracyxgbboost)
# print("Classification Report:\n", xgboostreport)

# xgboostcm = confusion_matrix(y_test, y_predxgboost)

# print("Confusion Matrix:")
# print(xgboostcm)

In [154]:
# from sklearn.svm import SVC


# # Assuming X_train, X_test, y_train, and y_test are your training and testing data
# # Create an instance of the SVC class
# Svcmodel = SVC()

# # Train the model
# Svcmodel.fit(X_train, y_train)

# # Make predictions
# y_predSVc = model.predict(X_test)

# # Calculate accuracy
# accuracySVC = accuracy_score(y_test, y_predSVc)

# SVCreport = classification_report(y_test, y_predSVc)
# print("Accuracy:", accuracySVC)

# print("Classification Report:\n", SVCreport)

# SVCcm = confusion_matrix(y_test, y_predSVc)

# print("Confusion Matrix:")
# print(SVCcm)


## prepro

> ID, Time, Card Number, merchant, category, Amount, firstName, lastName, trans_num, is_fraud

- time to datetime
- New Columns(fullNameday, cardhour, fullNamecard, fullNamesinglehour, fullNamehour, fullName, hour, month, year, day)
- New Column(time_diff)
- New Columns('countName', 'countNamehour', 'countNamesinglehour', 'countNamecard', 'countcardhour')
- New Column(dis)
- Drop Columns('Time','ID','merchant','trans_num','firstName','lastName')
- Drop Columns('Card Number','day','year','fullName','fullNamehour','fullNamesinglehour','fullNamecard','cardhour','fullNameday')
- Label Encoding('category', 'fullName', 'fullNamehour', 'fullNamesinglehour', 'fullNamecard', 'cardhour', 'fullNameday', 'time_diff')

